# Simulation scenarios

* Simulation basics
* Verification: Waveforms and assertions
* Writing portable simulations


## Simulation basics

Once a design is conceived, its correct functionality in conjunction with a known good template or routine has to be verified. This is typically done using a (virtual) test bench that tests the unit under test ('UUT') against external stimuli.

A typical test bench has one main stimulus routine defining a *sequence* of signal events.

Since cyrite does not provide a built-in simulator, it normally creates HDL output which is in turn fed to an external simulator such as GHDL or ICARUS. Again, HDL code is generated for the stimulus sequence.

There are several ways to instance a simulation. Let's start with a simple synchronous unit first:

In [1]:
import sys
sys.path.insert(0, '../..')

In [2]:
from cyhdl import *

Bool = Signal.Type(bool)

@block
def unit(clk : ClkSignal, en: Bool, a: Signal, q : Signal.Output):
    @always(clk.posedge)
    def worker():
        if en:
            q.next = ~a
    
    return worker

### Simple block test bench

A `@block` containing several stimuli can function as a test bench.
Once no more stimuli are present, the simulation is typically halted.

Because all event driven processes or sequences are translated to IRL generator notation internally, you can not use 'yield from' statements in the `@sequence` main stimulus. However, you can make use of `@hdlmacro`s for language-based simulators.

In [3]:
@block
def tb_unit(clkname):
    clk = ClkSignal(name=clkname)
    en = Bool()
    a, q = [ Signal(intbv()[8:]) for _ in range(2) ]
    
    uut = unit(clk, en, a, q)
    
    @always(delay(4))
    def clkgen():
        clk.next = ~clk
        
    @sequence
    def main():
        en.next = False
        yield delay(10)
        yield clk.negedge
        en.next = True
        a.next = 0xaa
        yield clk.negedge
        assert q == 0x55
        yield delay(100)
        
        raise StopSimulation # Terminate without error
        
    
    return instances()

In [4]:
clkname = 'clk'
tb = tb_unit(clkname)

s = Simulator(targets.VHDL)
s.run(tb, 80, wavetrace = "tb.vcd")

../../myirl/kernel/components.py:111: UserWarning: Fallback: Pass through other argument for arg clkname (<class 'str'>)
  base.warnings.warn(msg)


 Writing 'unit' to file /tmp/myirl_tb_unit_vjxf6gzm/unit.vhdl 
 Writing 'tb_unit' to file /tmp/myirl_tb_unit_vjxf6gzm/tb_unit.vhdl 
 DEBUG: omit `clkname` from interface, (passthrough type <class 'str'>) 
 Creating library file module_defs.vhdl 


0

When using GHDL, a VCD file with name given by the `wavetrace` parameter is created. To display this file in the notebook, we have to manually import a few wave drawing modules.

The wave utility requires a sample clock, whose clock name must be specified. In this case, it has to match the master clock's name.

In [5]:
from cyrite import waveutils
waveutils.draw_wavetrace(tb, 'tb.vcd', clkname)

## Specific test bench

A test bench can also be specific to a target and not be used with other simulators.
In this case, an extra decorator is prepended to the `@block` function, which specifies the simulator to use. The simulator has a default_target property which is used as elaboration target for the intermediate output.

In [6]:
from cyrite.simulation import sim, icarus, ghdl

@sim.testbench(icarus.ICARUS, 'ns')
@block
def tb_unit2(clkname):
    clk = ClkSignal(name=clkname)
    en = Bool()
    a, q = [ Signal(intbv()[8:]) for _ in range(2) ]
    
    uut = unit(clk, en, a, q)
    
    @always(delay(4))
    def clkgen():
        clk.next = ~clk
        
    @sequence
    def main():
        en.next = False
        yield delay(10)
        yield clk.negedge
        en.next = True
        a.next = 0xaa
        yield clk.negedge
        assert q == 0x55
        yield delay(100)
        
        raise StopSimulation # Terminate without error
        
    
    return instances()


As this is a specific test bench class bound to a simulator, it is the test bench instance that is '.run':

In [7]:
tb = tb_unit2('clk1')
tb.run(200)

../../myirl/kernel/components.py:111: UserWarning: Fallback: Pass through other argument for arg clkname (<class 'str'>)
  base.warnings.warn(msg)


 Module tb_unit2: Existing instance unit, rename to unit_1 
 Writing 'unit_1' to file /tmp/unit_1.v 
DEBUG Fallback wire for clk1
 Writing 'tb_unit2' to file /tmp/tb_unit2.v 
 DEBUG: omit `clkname` from interface, (passthrough type <class 'str'>) 
 Note: Changing library path prefix to /tmp/ 
 Creating library file /tmp/module_defs.v 
DEBUG FILES ['/tmp/unit_1.v', '/tmp/tb_unit2.v']
==== COSIM stdout ====
VCD info: dumpfile tb_unit2.vcd opened for output.
Stop Simulation



0

When Co-simulation is used, a test bench actually does **not** translate into IRL, as no HDL is created. In this case, a `@sim.testbench` decorator can occur without a `@block` notation.

Then, the python code is actually executed 'natively'. This is however advanced practise. Examples are found in the CXXRTL Co-Simulation tests. However, the recommended way is to create a portable test bench as shown below.

## Portable test bench

If a test bench should be run with several different simulator back ends, either for HDL targets or with Co-Simulation, the following derivation from a `cyrite_factory.Module` helps to create output for various architectures.

Here, the hardware generators are decorated by `@self.always` instead of `@always` and so forth. The reason is that these are depending on the target architecture:
*   HDL-Simulation output: All is transpiled to the target HDL
*   Co-Simulation: Hardware entities are transpiled, testbench processes are executed natively.

In [8]:
class TestDesign(cyrite_factory.Module):
    
    def __init__(self, name, arch, clktoggle_period):
        super().__init__(name, arch)
        self.clkperiod_half = clktoggle_period
    
    @cyrite_factory.testbench('ns')
    def tb_unitx(self):
        clk = self.ClkSignal(name='clk')
        en = self.Signal(bool())
        a, q = [ self.Signal(intbv()[8:]) for _ in range(2) ]

        uut = unit(clk, en, a, q)

        @self.always(delay(self.clkperiod_half))
        def clkgen():
            clk.next = ~clk

        @self.sequence
        def main():
            en.next = False
            yield delay(10)
            yield clk.negedge
            en.next = True
            a.next = 0xaa
            yield clk.negedge
            assert q == 0x55
            yield delay(100)

            raise StopSimulation # Terminate without error


        return instances()        

This test bench is portable among all three simulator architectures below. However, there are differences in the output, as you can see from the wave trace. The simulation will pass for all, though.

In [9]:
from yosys.simulator import CXXRTL

SIMULATOR = icarus.ICARUS
# SIMULATOR = ghdl.GHDL
# SIMULATOR = CXXRTL

design = TestDesign('test', SIMULATOR,
                    clktoggle_period = 3)
tb = design.tb_unitx()

tb.run(200, debug = False, wavetrace = True)


 Declare obj 'tb_unitx' in context '(TestDesign 'test')'(<class '__main__.TestDesign'>) 
 Module test: Existing instance unit, rename to unit_2 
 Insert unit unitu_1u_1u_8u_8 
 Insert unit tb_unitx_obj_TestDesign 
 Writing 'unit_2' to file /tmp/unit_2.v 
DEBUG Fallback wire for clk
 Writing 'tb_unitx' to file /tmp/tb_unitx.v 
 DEBUG: omit `self` from interface, (passthrough type <class '__main__.TestDesign'>) 
 Creating library file module_defs.v 


../../cyrite/simulation/icarus.py:35: UserWarning: Ignoring wavetrace argument for Verilog simulator
  warnings.warn("Ignoring wavetrace argument for Verilog simulator")


0

In [10]:
waveutils.draw_wavetrace(tb, 'tb_unitx.vcd', 'clk')

### Restrictions and pitfalls

Because `@self.sequence` is run in the co-simulation context when using CXXRTL, some hardware generator specific constructs such as `@hdlmacro` can not just be called like in their true hardware counterparts.

On the other hand, sequential minded macros can not be instanced in hardware, if they contain delays.

See [Ports (signal classes)](ports.ipynb) for more details.

### Simulator back end issues

Simulations may behave differently, depending on the simulator back ends. A simulation written for one back end may not behave the same on another. For example:

* An asserted signal depending on a previous assignment may be valid immediately or after a delta wait period
* Likewise, signals that depend on a synchronous clock may not be valid right after their driving clock event
* Some simulators like CXXRTL or the MyHDL simulator do not deal with undefined/uninitialized values

For portable simulators, a few thumb rules apply:

* Validate/assert signals on their opposite clock edge they are updated with
* Use context sensitive macros (`@cyrite_method.sequence` or `@cyrite_factory.Module::hdlmacro` (see [Factory class emulation](../notebooks/factory_class_arch.ipynb) to insert delays or toggle complex sequences

### CXXRTL extras

The CXXRTL back end in particular is driven by a simple CoSimulation layer that does not resolve complex circular asynchronous dependencies, neither is it sensitive to events or signal changes from within the simulated unit.

Also, it is a simulator for pure hardware entities that are either clock synchronous or asynchronous without any delay modelling. Therefore, units that drive a clock such a PLL can not be simulated within CXXRTL.

However, a safe assumption is: All elements that are accepted by yosys for synthesis typically translate to CXXRTL.

## Further reading

See also [Simulation API details](../notebooks/simulation.ipynb)